In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Imports loaded")

In [ ]:
# =============================================================================
# CONFIGURATION - EDIT THIS
# =============================================================================

# SECTORS TO SCAN
SECTORS_TO_SCAN = ['cybersecurity', 'uranium', 'quantum', 'space']

# SECTOR BASKETS
BASKETS = {
    'cybersecurity': ['CRWD', 'PANW', 'ZS', 'FTNT', 'OKTA', 'S', 'VRNS', 'TENB', 'CYBR', 'NET'],
    'uranium': ['UUUU', 'UEC', 'CCJ', 'LEU', 'DNN', 'NXE', 'URG', 'SMR'],
    'quantum': ['IONQ', 'RGTI', 'QBTS'],
    'space': ['RKLB', 'ASTS', 'LUNR'],
    'semi': ['NVTS', 'NXPI', 'SWKS', 'MRVL', 'ARM', 'NVDA', 'AMD', 'INTC', 'MU', 'TSM'],
}

# PARAMETERS (based on backtest results)
LEADER_THRESHOLD = 0.03  # leaders moved > 3% yesterday
COORDINATION_MIN = 4     # at least 4 stocks moving together
MOVE_THRESHOLD = 0.02    # 2% move to count as participating
LOOKBACK_DAYS = 5        # lookback for laggard identification

# TIME PERIOD
START_DATE = (datetime.now() - timedelta(days=60)).strftime('%Y-%m-%d')
END_DATE = datetime.now().strftime('%Y-%m-%d')

print("✓ Configuration set")
print(f"  Scanning sectors: {SECTORS_TO_SCAN}")
print(f"  Leader threshold: {LEADER_THRESHOLD*100:.0f}%")
print(f"  Coordination min: {COORDINATION_MIN} stocks")
print(f"  Date range: {START_DATE} to {END_DATE}")

In [ ]:
# =============================================================================
# DATA LOADING
# =============================================================================

def load_basket_data(tickers, start=START_DATE, end=END_DATE):
    data = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start, end=end, progress=False)
            if len(df) > 0:
                df['Returns'] = df['Adj Close'].pct_change()
                df['Ticker'] = ticker
                data[ticker] = df
        except Exception as e:
            pass
    return data

def load_sector(sector_name):
    if sector_name in BASKETS:
        return load_basket_data(BASKETS[sector_name])
    return {}

print("✓ Data loading functions ready")

In [ ]:
# =============================================================================
# SCANNER FUNCTIONS
# =============================================================================

def scan_yesterday(sector_data, threshold=0.02):
    """
    What happened yesterday? Any leader signals?
    """
    results = []
    
    for ticker, df in sector_data.items():
        if len(df) > 0:
            yesterday = df.iloc[-1]
            
            if len(df) > 20:
                avg_volume = df['Volume'].rolling(20).mean().iloc[-1]
                volume_ratio = yesterday['Volume'] / avg_volume if avg_volume > 0 else 0
            else:
                avg_volume = df['Volume'].mean()
                volume_ratio = yesterday['Volume'] / avg_volume if avg_volume > 0 else 0
            
            results.append({
                'ticker': ticker,
                'yesterday_return': yesterday['Returns'],
                'yesterday_close': yesterday['Adj Close'],
                'yesterday_volume': yesterday['Volume'],
                'volume_ratio': volume_ratio,
                'triggered': yesterday['Returns'] > threshold
            })
    
    results_df = pd.DataFrame(results)
    return results_df.sort_values('yesterday_return', ascending=False)

def check_coordination_today(sector_data, threshold=0.02):
    """
    How many stocks moved together yesterday?
    """
    up_movers = 0
    down_movers = 0
    total = 0
    
    for ticker, df in sector_data.items():
        if len(df) > 0:
            ret = df.iloc[-1]['Returns']
            if not pd.isna(ret):
                total += 1
                if ret > threshold:
                    up_movers += 1
                elif ret < -threshold:
                    down_movers += 1
    
    return {
        'up_movers': up_movers,
        'down_movers': down_movers,
        'total': total,
        'coordination_score': up_movers - down_movers,
        'bullish_pct': up_movers / total if total > 0 else 0
    }

def find_laggards(sector_data, lookback=5):
    """
    Find stocks that haven't moved as much as peers recently.
    """
    returns = {}
    
    for ticker, df in sector_data.items():
        if len(df) >= lookback:
            period_return = (df.iloc[-1]['Adj Close'] / df.iloc[-lookback]['Adj Close']) - 1
            returns[ticker] = period_return
    
    if not returns:
        return pd.DataFrame()
    
    returns_df = pd.DataFrame.from_dict(returns, orient='index', columns=['return'])
    returns_df = returns_df.sort_values('return')
    
    avg_return = returns_df['return'].mean()
    returns_df['vs_avg'] = returns_df['return'] - avg_return
    returns_df['is_laggard'] = returns_df['return'] < avg_return
    
    return returns_df

print("✓ Scanner functions ready")

In [ ]:
# =============================================================================
# MORNING SCAN - RUN THIS BEFORE MARKET OPEN
# =============================================================================

def morning_scan():
    """
    Full morning scan across all sectors.
    """
    print("="*60)
    print(f"🐺 WOLF PACK MORNING SCAN")
    print(f"   {datetime.now().strftime('%Y-%m-%d %H:%M')}")
    print("="*60)
    
    scan_summary = []
    
    for sector_name in SECTORS_TO_SCAN:
        print(f"\n{'='*40}")
        print(f"SECTOR: {sector_name.upper()}")
        print(f"{'='*40}")
        
        # Load fresh data
        print(f"Loading data...")
        sector_data = load_sector(sector_name)
        
        if len(sector_data) == 0:
            print(f"✗ No data loaded for {sector_name}")
            continue
        
        print(f"✓ Loaded {len(sector_data)} tickers\n")
        
        # Yesterday's action
        yesterday_df = scan_yesterday(sector_data, LEADER_THRESHOLD)
        
        # Leaders triggered?
        triggered = yesterday_df[yesterday_df['triggered']]
        if len(triggered) > 0:
            print(f"🚨 LEADER SIGNALS (moved > {LEADER_THRESHOLD*100:.0f}%):")
            for _, row in triggered.iterrows():
                print(f"  {row['ticker']:6s}: +{row['yesterday_return']*100:5.2f}% "
                      f"(${row['yesterday_close']:.2f}, vol: {row['volume_ratio']:.1f}x)")
        
        # Coordination
        coord = check_coordination_today(sector_data, MOVE_THRESHOLD)
        print(f"\n📊 COORDINATION:")
        print(f"  Up movers: {coord['up_movers']}/{coord['total']} ({coord['bullish_pct']*100:.0f}%)")
        print(f"  Coordination score: {coord['coordination_score']}")
        
        signal_strength = "STRONG" if coord['coordination_score'] >= COORDINATION_MIN else "WEAK"
        if coord['coordination_score'] >= COORDINATION_MIN:
            print(f"  ✅ HIGH COORDINATION - {signal_strength} sector rotation IN")
        elif coord['coordination_score'] <= -COORDINATION_MIN:
            print(f"  🔻 NEGATIVE COORDINATION - {signal_strength} sector rotation OUT")
        
        # Laggards
        laggards = find_laggards(sector_data, LOOKBACK_DAYS)
        if len(laggards) > 0:
            print(f"\n🎯 LAGGARDS ({LOOKBACK_DAYS}-day underperformers):")
            laggard_list = laggards[laggards['is_laggard']].head(3)
            if len(laggard_list) > 0:
                for ticker, row in laggard_list.iterrows():
                    print(f"  {ticker:6s}: {row['return']*100:+5.2f}% (vs avg: {row['vs_avg']*100:+5.2f}%)")
        
        # Top performers
        if len(laggards) > 0:
            print(f"\n🔥 LEADERS ({LOOKBACK_DAYS}-day top performers):")
            leader_list = laggards.tail(3).iloc[::-1]
            for ticker, row in leader_list.iterrows():
                print(f"  {ticker:6s}: {row['return']*100:+5.2f}%")
        
        # Store summary
        scan_summary.append({
            'sector': sector_name,
            'leader_signals': len(triggered),
            'coordination_score': coord['coordination_score'],
            'signal_strength': signal_strength,
            'up_movers': coord['up_movers'],
            'total_stocks': coord['total']
        })
    
    # Overall summary
    print("\n" + "="*60)
    print("📋 SCAN SUMMARY")
    print("="*60)
    
    summary_df = pd.DataFrame(scan_summary)
    if len(summary_df) > 0:
        summary_df = summary_df.sort_values('coordination_score', ascending=False)
        print(summary_df.to_string(index=False))
        
        print("\n🎯 TOP SETUPS:")
        top_sectors = summary_df[summary_df['coordination_score'] >= COORDINATION_MIN]
        if len(top_sectors) > 0:
            for _, row in top_sectors.iterrows():
                print(f"  {row['sector'].upper()}: {row['up_movers']}/{row['total_stocks']} movers, "
                      f"{row['leader_signals']} leader signals")
        else:
            print("  No strong setups detected (coordination < threshold)")
    
    print("\n" + "="*60)
    print("✅ SCAN COMPLETE")
    print("="*60)

print("✓ Morning scan function ready")

In [ ]:
# =============================================================================
# RUN IT - EXECUTE THE SCAN
# =============================================================================

morning_scan()

---

## 📊 HOW TO USE THIS SCANNER

### Timing
- **Before market open (9:00 AM):** Scan yesterday's close data to plan today's trades
- **After market close (4:00 PM):** Scan today's action to prepare for tomorrow

### Interpretation

**Leader Signals:**
- 1-2 leaders = individual moves (wait for confirmation)
- 3+ leaders = sector heating up (enter laggards)

**Coordination Score:**
- **Score ≥ 4:** Strong bullish rotation INTO sector
- **Score 2-3:** Moderate interest, wait for confirmation
- **Score ≤ -4:** Strong bearish rotation OUT of sector

**Up Movers %:**
- **> 60%:** Sector moving together (coordination)
- **40-60%:** Mixed action (no clear trend)
- **< 40%:** Sector weakening

### Action Plan

**If high coordination + leader signals:**
1. Buy top 3 laggards from that sector
2. Set stop loss at -5%
3. Hold while coordination stays high
4. Exit when coordination drops below 3

**If negative coordination:**
1. Exit all positions in that sector
2. Wait for coordination to turn positive
3. Re-enter when 4+ movers detected

### Customization

**Add more sectors:**
```python
BASKETS['ai_infra'] = ['NVDA', 'AMD', 'MU', 'AVGO', 'MRVL']
SECTORS_TO_SCAN = ['cybersecurity', 'uranium', 'quantum', 'space', 'ai_infra']
```

**Adjust thresholds:**
```python
LEADER_THRESHOLD = 0.04  # 4% instead of 3%
COORDINATION_MIN = 5     # 5 stocks instead of 4
```

🐺 **Scan. Identify. Execute. The pack hunts at dawn.**